In [ ]:
# !pip install git+https://git@github.com/SadeghKrmi/pernorm.git
# !pip install git+https://git@github.com/SadeghKrmi/zirneshane.git
# !pip install git+https://git@github.com/SadeghKrmi/vaguye.git
# !pip install git+https://git@github.com/SadeghKrmi/hamnevise.git


In [1]:
from zirneshane import HybridZirneshanModel

# Load model (auto-downloads from HuggingFace)
zmodel = HybridZirneshanModel.load()

📥 Downloading default model from HuggingFace...


zirneshan-word-char-parsbert-embedding-c(…):   0%|          | 0.00/657M [00:00<?, ?B/s]

📂 Loading model from: /root/.cache/huggingface/hub/models--SadeghK--zirneshane/snapshots/fa0b943ba9024e24fee59b9840daf29b89960ce8/zirneshan-word-char-parsbert-embedding-classifier-v2.0.pt


config.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/654M [00:00<?, ?B/s]

✅ Model loaded successfully!
   Epoch: 10
   F1 Score: 0.7417


In [2]:
from pernorm import PersianNormalizer
normer = PersianNormalizer()

In [3]:
from hamnevise import HamneviseModel
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# Load model + tokenizer
hmodel, tokenizer = HamneviseModel.load(device=device)

# Cache the set of words that need hamnevise disambiguation
hamnevise_words = set(hmodel.word2idx.keys())

# disambiguate all ambiguous words in a sentence
sentence = "اشکال در سیستم گرمایش، در سمت چپ، یک اشکال اساسی است که باعث سرد شدن ساختمان شد که باعث شد آن را سرد کند."

new_sentence, _ = hmodel.disambiguate(sentence, tokenizer)
print("Original:      ", sentence)
print("Disambiguated: ", new_sentence)

hamnevise-persian-word-disambigution-v1.(…):   0%|          | 0.00/673M [00:00<?, ?B/s]

tokenizer-config-v1.0.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/440 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/654M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/654M [00:00<?, ?B/s]


🏗️  Model Architecture:
   - Shared encoder: HooshvareLab/bert-fa-base-uncased
   - Word-specific heads: 138
   - Character vocab: 67
📦 Files ready
  Model : /root/.cache/huggingface/hub/models--SadeghK--Hamnevise/snapshots/f89b2c2ac747866768c4cdd061eb6eb759e47a17/hamnevise-persian-word-disambigution-v1.0.pt
  Config: /root/.cache/huggingface/hub/models--SadeghK--Hamnevise/snapshots/f89b2c2ac747866768c4cdd061eb6eb759e47a17/tokenizer-config-v1.0.json
Original:       اشکال در سیستم گرمایش، در سمت چپ، یک اشکال اساسی است که باعث سرد شدن ساختمان شد که باعث شد آن را سرد کند.
Disambiguated:  اِشکال در سیستم گرمایش، در سَمت چپ، یک اَشکال اساسی است که باعث سرد شدن ساختمان شد که باعث شد آن را سرد کَند.


In [4]:
from google.colab import output
output.disable_custom_widget_manager()

In [10]:
def _needs_hamnevise(text):
  """Check if any word in text requires hamnevise disambiguation."""
  words = text.split()
  return any(word in hamnevise_words for word in words)

def _process_single(text):
  """Process a single chunk of text through the pipeline."""
  output = {}

  # Step 1: Normalize
  norm = normer.normalize(text)

  # Step 2: Hamnevise (only if needed)
  if _needs_hamnevise(norm):
      ham, _ = hmodel.disambiguate(norm, tokenizer=tokenizer)
  else:
      # Skip hamnevise if no relevant words found
      ham = norm

  # Step 3: Zirnešān
  zir = zmodel.predict(ham)

  return zir, norm, ham

In [11]:
# start_line = 143800 + 219184 + 125885 + 142460 + 89900 # <-- change this

# input_path = "./wikipedia-fa.txt"
# output_path = "./wikipedia-fa-out-part7.txt"

# with open(input_path, "r", encoding="utf-8") as infile, \
#      open(output_path, "w", encoding="utf-8") as outfile:

#     for i, line in enumerate(infile, 1):  # i starts at 1
#         if i < start_line:
#             continue  # skip lines until start_line

#         result, _, _ = _process_single(line)
#         outfile.write(result + "\n")

#         if i % 10000 == 0:
#             print(f"Processed {i} lines")


In [ ]:
input_path = "./metadata-styletts2.txt"
output_path = "./metadata-styletts2-norm-ham-zir.txt"

with open(input_path, "r", encoding="utf-8") as infile, \
     open(output_path, "w", encoding="utf-8") as outfile:

    for i, line in enumerate(infile, 1):
        text = line.strip().split("|")[-1]
        zir, norm, ham = _process_single(text)

        # print(line.rstrip('\n') + "|" + norm + "|" + ham + "|" + zir + "\n")
        # break

        outfile.write(line.rstrip('\n') + "|" + norm + "|" + ham + "|" + zir + "\n")

        if i % 10000 == 0:
            print(f"Processed {i} lines")

In [ ]:
#  یُمن